<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/K_spell_check%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_C_inc.csv	   df_K_inc.csv     spell_check
'Colab Notebooks'	'# df_G_inc.csv'   df_P_inc.csv
 C_ori_model.pt		 df_G_inc.csv	  'My Drive'
'답안 작성용 파일.csv'	 df_J_inc.csv	   Q_inc_model.pt


In [4]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [5]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [6]:
df=df[(df['digit_1'] =='K')] #조건에 맞는 데이터 출력

In [7]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000377,K,66,662,"선박화물,해사관련","검량,검정",손해사정 서비스
1,id_0000397,K,66,662,보험대리점,보험사업자를 대리 알선,"생명,손해 보험"
2,id_0000414,K,64,641,신협,NaN,NaN
3,id_0000526,K,65,653,사무실에서,직원들에게,복지활동
4,id_0000631,K,64,641,신협,고객대상 금융서비스,금융서비스
...,...,...,...,...,...,...,...
10373,id_0999360,K,66,662,보험대리점,보험사업자를 대리 알선,보험계약 대리 중개
10374,id_0999365,K,66,662,보험대리점,보험사업자를 대리 알선,"보험계약, 대리중개"
10375,id_0999640,K,66,662,보험대리점,보험사업자를 대리 알선,보험계약 대리 중개
10376,id_0999831,K,66,662,보험대리점,"보험사업자를 대리,알선","보험계약 대리, 중개"


In [8]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000377,K,66,662,"선박화물,해사관련 검량,검정 손해사정 서비스"
1,id_0000397,K,66,662,"보험대리점 보험사업자를 대리 알선 생명,손해 보험"
2,id_0000414,K,64,641,신협
3,id_0000526,K,65,653,사무실에서 직원들에게 복지활동
4,id_0000631,K,64,641,신협 고객대상 금융서비스 금융서비스
...,...,...,...,...,...
10373,id_0999360,K,66,662,보험대리점 보험사업자를 대리 알선 보험계약 대리 중개
10374,id_0999365,K,66,662,"보험대리점 보험사업자를 대리 알선 보험계약, 대리중개"
10375,id_0999640,K,66,662,보험대리점 보험사업자를 대리 알선 보험계약 대리 중개
10376,id_0999831,K,66,662,"보험대리점 보험사업자를 대리,알선 보험계약 대리, 중개"


In [9]:
df['digit_3'].value_counts()

641    4113
662    1983
651    1715
661    1211
649    1018
653     171
642     165
652       2
Name: digit_3, dtype: int64

## 맞춤법 교정

In [10]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-v4d8c46o
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-v4d8c46o
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=a25856af6ad4621fb6270e1259dababb2a54fab3dc591c6b15a13f397f6bfde4
  Stored in directory: /tmp/pip-ephem-wheel-cache-pawpzn01/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [11]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [12]:
df.to_csv('df_K_spellcheck.csv', index=False, encoding='utf-8-sig')